From the RFC guided example in the Bootcamp 3.2.5.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

First do the cleaning as outlined in the guided example.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

y2015 = pd.read_csv('LoanStats3d_1.csv', skipinitialspace=True, header=1)

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

y2015 = y2015[:-2]

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array([0.9794828 , 0.98069388, 0.98135879, 0.98176248, 0.97539777,
       0.97482783, 0.96055475, 0.98038424, 0.98005082, 0.98040659])

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [2]:
#Dropping anything related to payment amount or outstanding principal.
y2015.drop(y2015.filter(regex = 'pymnt_amnt').columns, axis = 1, inplace = True)
y2015.drop(y2015.filter(regex = 'bal').columns, axis = 1, inplace = True)

#Dropping useless columns by visual checking
y2015.drop(['policy_code', 'id', 'member_id', 'title'], 1, inplace=True)

#Correlation matrix
cor = y2015.corr()

#Dropping highly correlated columns
y2015.drop(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'out_prncp_inv',
                    'total_pymnt_inv', 'total_rec_prncp', 'collection_recovery_fee'], 1, inplace=True)

#Redo the RFC
X_new = y2015.drop('loan_status', 1)
Y_new = y2015['loan_status']
X_new = pd.get_dummies(X_new)
X_new = X_new.dropna(axis=1)

cross_val_score(rfc, X_new, Y_new, cv=10)

array([0.97440099, 0.9739498 , 0.97627698, 0.97717936, 0.96995963,
       0.9738067 , 0.95882115, 0.97608587, 0.97639347, 0.97534793])

In [6]:
X_new.head()

,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,14.85,379.39,48000.0,33.18,0.0,0.0,11.0,2.0,19.0,13668.88,...,0,0,0,0,0,1,0,0,0,0
1,7.49,298.58,60000.0,22.44,0.0,0.0,7.0,0.0,9.0,6635.69,...,0,0,0,0,0,1,0,0,0,0
2,7.49,777.55,109000.0,26.02,0.0,1.0,9.0,0.0,19.0,0.00,...,0,0,0,0,0,1,0,0,0,0
3,6.49,858.05,92000.0,21.60,0.0,0.0,16.0,0.0,24.0,19263.77,...,0,0,0,0,0,1,0,0,0,0
4,19.89,320.99,55000.0,25.49,0.0,4.0,18.0,1.0,19.0,0.00,...,0,0,0,0,0,1,0,0,0,0


### PCA


In [7]:
X_pca = StandardScaler().fit_transform(X_new)
pca = PCA(n_components=X_new.shape[1])
Y_pca = pca.fit_transform(X_pca)

pca.explained_variance_ratio_

array([4.82421325e-02, 2.83792173e-02, 2.59150263e-02, 2.27811923e-02,
       1.91932906e-02, 1.83283848e-02, 1.44129925e-02, 1.37445698e-02,
       1.34217187e-02, 1.17987518e-02, 1.12829505e-02, 1.08482997e-02,
       1.07964311e-02, 1.02823048e-02, 1.01401163e-02, 9.65243120e-03,
       9.21513109e-03, 9.19739944e-03, 9.08605184e-03, 8.93690394e-03,
       8.90256295e-03, 8.73929062e-03, 8.72928177e-03, 8.63740438e-03,
       8.60986113e-03, 8.48094131e-03, 8.44605806e-03, 8.32851180e-03,
       8.26947724e-03, 8.18842558e-03, 8.14311445e-03, 8.06793272e-03,
       7.96605567e-03, 7.88993448e-03, 7.82451622e-03, 7.79754665e-03,
       7.73017353e-03, 7.70829015e-03, 7.59837558e-03, 7.51345090e-03,
       7.44613437e-03, 7.41308549e-03, 7.36460399e-03, 7.30983125e-03,
       7.22302976e-03, 7.16171334e-03, 7.09455117e-03, 7.05329392e-03,
       6.99664162e-03, 6.97934641e-03, 6.92611486e-03, 6.91383825e-03,
       6.89981418e-03, 6.85986866e-03, 6.82994494e-03, 6.81126543e-03,
      

Looks like the first 15 components at least explain much more than the rest. So I will just take the first 18 out.

In [8]:
df_components = pd.DataFrame(Y_pca)
pca_features = df_components.iloc[:, : 15]

X_new_pca = pca_features
Y_new_pca = y2015['loan_status']

cross_val_score(rfc, X_new_pca, Y_new_pca, cv=10)

array([0.89506305, 0.89720026, 0.90104723, 0.90634276, 0.90484445,
       0.90729043, 0.90790567, 0.90759695, 0.91155865, 0.91032157])